In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import urllib
import re
import sys
from glob import glob

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('/Users/tevansbarton/AAASideProjects/GMDraftTendencies/config')
sys.path.append('/Users/tevansbarton/AAASideProjects/GMDraftTendencies/src')
sys.path.append('/Users/tevansbarton/AAASideProjects/GMDraftTendencies/src/data')
sys.path.append('/Users/tevansbarton/AAASideProjects/GMDraftTendencies/src/processing')

In [4]:
import etl
import processing

In [5]:
TOP_PATH = os.environ['PWD']

In [6]:
with open(TOP_PATH + '/config/data-params.json') as fh:
    data_cfg = json.load(fh)

In [7]:
etl.get_data(**data_cfg)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [8]:
draft_2018 = pd.read_csv(TOP_PATH + '/data/raw/DRAFT_2018.csv')

In [13]:
draft_2018.describe().T

,count,mean,std,min,25%,50%,75%,max
Rnd,256.0,4.167969,1.980075,1.0,2.75,4.0,6.00,7.0
Pick,256.0,128.500000,74.045031,1.0,64.75,128.5,192.25,256.0
Age,256.0,22.226562,0.939043,20.0,22.00,22.0,23.00,26.0
To,233.0,2018.935622,0.245953,2018.0,2019.00,2019.0,2019.00,2019.0
AP1,256.0,0.027344,0.185859,0.0,0.00,0.0,0.00,2.0
PB,256.0,0.062500,0.258199,0.0,0.00,0.0,0.00,2.0
St,256.0,0.535156,0.755849,0.0,0.00,0.0,1.00,2.0
Approx Val CarAV,233.0,5.815451,5.683106,0.0,2.00,4.0,8.00,33.0
Approx Val DrAV,223.0,5.834081,5.681827,-1.0,2.00,4.0,8.00,33.0
G,233.0,22.502146,8.776990,1.0,16.00,25.0,30.00,32.0


In [132]:
executives = pd.read_csv(TOP_PATH + '/data/raw/EXECUTIVES.csv')

In [133]:
curr_execs = processing.get_current_executives_and_clean(executives)

In [134]:
curr_execs

,Person,Teams,From,To,Titles,Years Worked
0,Brian Gutekunst,Packers,2018,2019,General Manager,"[2018, 2019]"
1,Tom Telesco,Chargers,2013,2019,General Manager,"[2013, 2014, 2015, 2016, 2017, 2018, 2019]"
2,Mike Mayock,Raiders,2019,2019,General Manager,[2019]
3,John Elway,Broncos,2011,2019,"Director of Player Personnel, Executive VP of ...","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
4,John Dorsey,Chiefs,2013,2016,General Manager,"[2013, 2014, 2015, 2016]"
5,Brett Veach,Chiefs,2017,2019,General Manager,"[2017, 2018, 2019]"
6,Mike Brown,Bengals,1991,2019,Principal Owner/President/General Manager,"[1991, 1992, 1993, 1994, 1995, 1996, 1997, 199..."
7,Bill Belichick,Browns,1992,1995,Head Coach/de facto General Manager,"[1992, 1993, 1994, 1995]"
8,John Dorsey,Browns,2017,2019,General Manager,"[2017, 2018, 2019]"
9,Kevin Colbert,Steelers,2000,2019,"General Manager, VP & General Manager","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."


In [137]:
analysis_df = processing.combine_drafts_and_executives(full_df, curr_execs)

In [163]:
round_picks_by_gm = analysis_df.groupby(['GM', 'Rnd'])['Pos'].value_counts().unstack().fillna(0)

In [164]:
round_picks_by_gm.loc['Dave Gettleman'].loc[1]

Pos
C      0.0
CB     1.0
DB     0.0
DE     0.0
DL     0.0
DT     3.0
FB     0.0
G      0.0
ILB    0.0
K      0.0
LB     0.0
LS     0.0
NT     0.0
OL     0.0
OLB    1.0
P      0.0
QB     1.0
RB     2.0
S      0.0
T      0.0
TE     0.0
WR     1.0
Name: 1, dtype: float64

In [165]:
round_picks = analysis_df.groupby('Rnd')['Pos'].value_counts().unstack().fillna(0)

In [166]:
round_picks

Pos,C,CB,DB,DE,DL,DT,FB,G,ILB,K,...,NT,OL,OLB,P,QB,RB,S,T,TE,WR
Rnd,,,,,,,,,,,,,,,,,,,,,
1,5.0,14.0,13.0,21.0,0.0,22.0,0.0,9.0,4.0,0.0,...,0.0,0.0,9.0,0.0,17.0,11.0,8.0,25.0,6.0,20.0
2,7.0,25.0,12.0,17.0,0.0,13.0,0.0,9.0,2.0,1.0,...,1.0,0.0,10.0,0.0,6.0,17.0,10.0,16.0,15.0,27.0
3,5.0,20.0,17.0,22.0,0.0,21.0,0.0,13.0,4.0,0.0,...,0.0,5.0,7.0,0.0,9.0,21.0,10.0,13.0,11.0,31.0
4,7.0,12.0,18.0,21.0,0.0,14.0,0.0,19.0,6.0,1.0,...,1.0,3.0,5.0,1.0,10.0,22.0,16.0,8.0,13.0,28.0
5,3.0,19.0,27.0,13.0,0.0,14.0,0.0,14.0,2.0,4.0,...,1.0,3.0,7.0,5.0,7.0,20.0,9.0,13.0,14.0,26.0
6,8.0,19.0,16.0,15.0,0.0,17.0,7.0,12.0,5.0,3.0,...,1.0,8.0,5.0,2.0,10.0,26.0,15.0,14.0,8.0,36.0
7,6.0,12.0,27.0,27.0,4.0,23.0,2.0,7.0,4.0,5.0,...,1.0,5.0,11.0,2.0,11.0,26.0,6.0,15.0,19.0,28.0


In [172]:
curr_execs

,Person,Teams,From,To,Titles,Years Worked
0,Brian Gutekunst,Packers,2018,2019,General Manager,"[2018, 2019]"
1,Tom Telesco,Chargers,2013,2019,General Manager,"[2013, 2014, 2015, 2016, 2017, 2018, 2019]"
2,Mike Mayock,Raiders,2019,2019,General Manager,[2019]
3,John Elway,Broncos,2011,2019,"Director of Player Personnel, Executive VP of ...","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
4,John Dorsey,Chiefs,2013,2016,General Manager,"[2013, 2014, 2015, 2016]"
5,Brett Veach,Chiefs,2017,2019,General Manager,"[2017, 2018, 2019]"
6,Mike Brown,Bengals,1991,2019,Principal Owner/President/General Manager,"[1991, 1992, 1993, 1994, 1995, 1996, 1997, 199..."
7,Bill Belichick,Browns,1992,1995,Head Coach/de facto General Manager,"[1992, 1993, 1994, 1995]"
8,John Dorsey,Browns,2017,2019,General Manager,"[2017, 2018, 2019]"
9,Kevin Colbert,Steelers,2000,2019,"General Manager, VP & General Manager","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."


In [170]:
analysis_df['Pos'].value_counts()

WR     196
RB     143
LB     138
DE     136
DB     130
DT     124
CB     121
T      104
TE      86
G       83
S       74
QB      70
OLB     54
C       41
ILB     27
OL      24
K       14
P       10
FB       9
LS       5
NT       5
DL       4
Name: Pos, dtype: int64

In [171]:
analysis_df[analysis_df['Pos'] == 'DB']

,Rnd,Pick,Tm,Player,Pos,Age,Played To,AP1,PB,St,...,YEAR,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,GM
21,2,39,Rams,Janoris Jenkins,DB,23.0,2019.0,0,1,8,...,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Les Snead
23,3,65,Rams,Trumaine Johnson,DB,22.0,2019.0,0,0,5,...,2012,74.0,204.0,4.50,35.5,19.0,122.0,7.20,4.15,Les Snead
31,3,71,Rams,T.J. McDonald,DB,22.0,2018.0,0,0,6,...,2013,74.0,219.0,4.59,40.0,19.0,131.0,6.89,4.20,Les Snead
34,5,149,Rams,Brandon McGee,DB,22.0,2015.0,0,0,0,...,2013,71.0,193.0,4.40,33.5,14.0,119.0,6.71,4.18,Les Snead
38,2,41,Rams,Lamarcus Joyner,DB,23.0,2019.0,0,0,3,...,2014,68.0,184.0,4.55,37.5,14.0,124.0,7.26,4.40,Les Snead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,7,216,Panthers,D.J. Campbell,DB,23.0,2013.0,0,0,0,...,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marty Hurney
2713,4,128,Panthers,Tre Boston,DB,22.0,2019.0,0,0,4,...,2014,72.0,204.0,4.58,35.0,18.0,116.0,7.04,4.31,Marty Hurney
2714,4,128,Panthers,Tre Boston,DB,22.0,2019.0,0,0,4,...,2014,72.0,204.0,4.58,35.0,18.0,116.0,7.04,4.31,Dave Gettleman
2715,5,148,Panthers,Bene Benwikere,DB,23.0,2018.0,0,0,1,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marty Hurney


Need to map DL : DE